In [22]:
# Cell 2: Imports and helper functions
import pandas as pd
import numpy as np
from random import choice, randint, seed

seed(42)
np.random.seed(42)

def make_student_id(n):
    return f"UG2025{n:04d}"

def generate_name(i):
    first = ['Kwame','Akosua','Yaa','Kofi','Ama','Yaw','Abena','Kojo','Esi','Nana']
    last = ['Mensah','Boateng','Owusu','Amankwa','Amartey','Yeboah','Asare','Quaye','Kumah','Ofori']
    return f"{choice(first)} {choice(last)} {i}"

In [23]:
# Cell 3: Generate 60 students with aggregates (lower aggregate is better)
num_students = 60
students = []
for i in range(1, num_students+1):
    sid = make_student_id(i)
    name = generate_name(i)
    # WASSCE-like aggregate, 6 (best) to 30 (worst)
    aggregate = randint(6, 24)
    students.append({'StudentID': sid, 'Name': name, 'Aggregate': aggregate})

df_students = pd.DataFrame(students)
df_students.head()

,StudentID,Name,Aggregate
0,UG20250001,Akosua Mensah 1,14
1,UG20250002,Kofi Amankwa 2,10
2,UG20250003,Akosua Kumah 3,8
3,UG20250004,Nana Asare 4,7
4,UG20250005,Kwame Boateng 5,12


In [24]:
# Cell 4: Create programs database with cutoffs
programs_data = [
    {'ProgramID': 'UGP001', 'University': 'University of Ghana', 'Programme': 'Medicine and Surgery', 'Cutoff': 6},
    {'ProgramID': 'UGP002', 'University': 'University of Ghana', 'Programme': 'Law', 'Cutoff': 9},
    {'ProgramID': 'UGP003', 'University': 'University of Ghana', 'Programme': 'Biochemistry', 'Cutoff': 8},
    {'ProgramID': 'UGP004', 'University': 'KNUST', 'Programme': 'Computer Science', 'Cutoff': 10},
    {'ProgramID': 'UGP005', 'University': 'KNUST', 'Programme': 'Mechanical Engineering', 'Cutoff': 11},
    {'ProgramID': 'UGP006', 'University': 'UG Bears', 'Programme': 'Business Administration', 'Cutoff': 12},
    {'ProgramID': 'UGP007', 'University': 'UM', 'Programme': 'Nursing', 'Cutoff': 13},
    {'ProgramID': 'UGP008', 'University': 'UCC', 'Programme': 'Psychology', 'Cutoff': 14},
    {'ProgramID': 'UGP009', 'University': 'UENR', 'Programme': 'Civil Engineering', 'Cutoff': 11},
    {'ProgramID': 'UGP010', 'University': 'Valley U', 'Programme': 'Horticulture', 'Cutoff': 15}
]

df_programs = pd.DataFrame(programs_data)
df_programs = df_programs.sort_values(by='Cutoff')
df_programs

,ProgramID,University,Programme,Cutoff
0,UGP001,University of Ghana,Medicine and Surgery,6
2,UGP003,University of Ghana,Biochemistry,8
1,UGP002,University of Ghana,Law,9
3,UGP004,KNUST,Computer Science,10
4,UGP005,KNUST,Mechanical Engineering,11
8,UGP009,UENR,Civil Engineering,11
5,UGP006,UG Bears,Business Administration,12
6,UGP007,UM,Nursing,13
7,UGP008,UCC,Psychology,14
9,UGP010,Valley U,Horticulture,15


In [25]:
# Cell 5: Assign first/second/third choices randomly for each student
choices_list = df_programs['Programme'].tolist()

choices = []
for sid in df_students['StudentID']:
    ch = np.random.choice(choices_list, size=3, replace=False)
    choices.append({'StudentID': sid, 'Choice1': ch[0], 'Choice2': ch[1], 'Choice3': ch[2]})

df_choices = pd.DataFrame(choices)
df_choices.head()

,StudentID,Choice1,Choice2,Choice3
0,UG20250001,Psychology,Biochemistry,Civil Engineering
1,UG20250002,Medicine and Surgery,Biochemistry,Psychology
2,UG20250003,Horticulture,Law,Medicine and Surgery
3,UG20250004,Biochemistry,Nursing,Business Administration
4,UG20250005,Biochemistry,Civil Engineering,Mechanical Engineering


In [26]:
# Cell 6: Evaluate each applicant vs their three choices and add suggested programme + fee-paying suggestions
def evaluate_applicant(aggregate, c1, c2, c3, programs_df, fee_margin=2):
    # Find cutoffs
    def cutoff_for(programme):
        row = programs_df.loc[programs_df['Programme'] == programme]
        if not row.empty:
            return int(row['Cutoff'].values[0])
        else:
            return 999

    c1_cut = cutoff_for(c1)
    c2_cut = cutoff_for(c2)
    c3_cut = cutoff_for(c3)

    # Decision logic using if / elif / else
    if aggregate <= c1_cut:
        return 'Yes', 'No', 'No', c1, 'Direct Admission'
    elif aggregate <= c2_cut:
        return 'No', 'Yes', 'No', c2, 'Direct Admission'
    elif aggregate <= c3_cut:
        return 'No', 'No', 'Yes', c3, 'Direct Admission'
    else:
        # Search for any programme student qualifies for (aggregate <= cutoff)
        qualified = programs_df.loc[programs_df['Cutoff'] >= aggregate]
        if not qualified.empty:
            suggested = qualified.sort_values(by='Cutoff').iloc[0]['Programme']
            return 'No', 'No', 'No', suggested, 'Direct Admission (other programme)'
        else:
            # Suggest fee-paying programmes where aggregate is within fee_margin points of cutoff
            fee_candidates = programs_df.loc[(programs_df['Cutoff'] + fee_margin) >= aggregate]
            # keep only programs where aggregate is worse than cutoff (needs fee-paying)
            fee_candidates = fee_candidates.loc[fee_candidates['Cutoff'] < aggregate]
            if not fee_candidates.empty:
                suggested = fee_candidates.sort_values(by='Cutoff').iloc[0]['Programme']
                return 'No', 'No', 'No', suggested, 'Fee-Paying Suggestion'
            else:
                return 'No', 'No', 'No', 'No matching programme', 'No Suggestion'

results = []
for _, row in df_students.merge(df_choices, on='StudentID').iterrows():
    f1, f2, f3, suggestion, comment = evaluate_applicant(row['Aggregate'], row['Choice1'], row['Choice2'], row['Choice3'], df_programs)
    results.append({'StudentID': row['StudentID'], 'FirstChoice': f1, 'SecondChoice': f2, 'ThirdChoice': f3, 'SuggestedProgramme': suggestion, 'SuggestionType': comment})

df_results = pd.DataFrame(results)
df_output = df_students.merge(df_choices, on='StudentID').merge(df_results, on='StudentID')
df_output.head()

,StudentID,Name,Aggregate,Choice1,Choice2,Choice3,FirstChoice,SecondChoice,ThirdChoice,SuggestedProgramme,SuggestionType
0,UG20250001,Akosua Mensah 1,14,Psychology,Biochemistry,Civil Engineering,Yes,No,No,Psychology,Direct Admission
1,UG20250002,Kofi Amankwa 2,10,Medicine and Surgery,Biochemistry,Psychology,No,No,Yes,Psychology,Direct Admission
2,UG20250003,Akosua Kumah 3,8,Horticulture,Law,Medicine and Surgery,Yes,No,No,Horticulture,Direct Admission
3,UG20250004,Nana Asare 4,7,Biochemistry,Nursing,Business Administration,Yes,No,No,Biochemistry,Direct Admission
4,UG20250005,Kwame Boateng 5,12,Biochemistry,Civil Engineering,Mechanical Engineering,No,No,No,Business Administration,Direct Admission (other programme)


In [27]:
# Cell 7: Export initial data (3 sheets) and results to Excel with styling
output_initial = 'applicants_input.xlsx'
with pd.ExcelWriter(output_initial, engine='xlsxwriter') as writer:
    df_students.to_excel(writer, sheet_name='Students', index=False)
    df_choices.to_excel(writer, sheet_name='Choices', index=False)
    df_programs.to_excel(writer, sheet_name='Programs', index=False)

output_results = 'admissions_results.xlsx'

# Build a summary for visualization
summary = {
    'Outcome': ['FirstChoice', 'SecondChoice', 'ThirdChoice', 'NoMatch'],
    'Count': [
        len(df_output.loc[df_output['FirstChoice']=='Yes']),
        len(df_output.loc[df_output['SecondChoice']=='Yes']),
        len(df_output.loc[df_output['ThirdChoice']=='Yes']),
        len(df_output.loc[(df_output['FirstChoice']=='No') & (df_output['SecondChoice']=='No') & (df_output['ThirdChoice']=='No')])
    ]
}
df_summary = pd.DataFrame(summary)

with pd.ExcelWriter(output_results, engine='xlsxwriter') as writer:
    df_output.to_excel(writer, sheet_name='Results', index=False)
    df_summary.to_excel(writer, sheet_name='Summary', index=False)

    # Add some basic formatting and chart (all within same xlsxwriter session)
    workbook  = writer.book
    ws_results = writer.sheets['Results']
    ws_summary = writer.sheets['Summary']
    fmt_yes = workbook.add_format({'font_color': 'green'})
    fmt_no  = workbook.add_format({'font_color': 'red'})
    ws_results.conditional_format('G2:G999', {'type': 'text', 'criteria': 'containing', 'value': 'Yes', 'format': fmt_yes})
    ws_results.conditional_format('G2:G999', {'type': 'text', 'criteria': 'containing', 'value': 'No', 'format': fmt_no})

    chart = workbook.add_chart({'type': 'column'})
    chart.add_series({'name': 'Outcomes', 'categories': '=Summary!$A$2:$A$5', 'values': '=Summary!$B$2:$B$5'})
    chart.set_title({'name': 'Admissions Outcomes'})
    ws_summary.insert_chart('D2', chart)

print('Created', output_initial, 'and', output_results)

Created applicants_input.xlsx and admissions_results.xlsx


In [28]:
# Cell 8: Add a summary sheet and a simple chart for visualization
summary = {
    'Outcome': ['FirstChoice', 'SecondChoice', 'ThirdChoice', 'NoMatch'],
    'Count': [
        len(df_output.loc[df_output['FirstChoice']=='Yes']),
        len(df_output.loc[df_output['SecondChoice']=='Yes']),
        len(df_output.loc[df_output['ThirdChoice']=='Yes']),
        len(df_output.loc[(df_output['FirstChoice']=='No') & (df_output['SecondChoice']=='No') & (df_output['ThirdChoice']=='No')])
    ]
}
df_summary = pd.DataFrame(summary)

# Summary and chart are already written in the main xlsxwriter session above; no append needed.
print('Summary already written in the primary write session; no append required')

Summary already written in the primary write session; no append required


### Notes
- Uses if / elif / else as required
- Student IDs are in the format UG20250000
- The dataset is fictitious; adjust cutoffs and program list to fit a real institution's requirements